In [1]:
import numpy as np
import pandas as pd

# Increase display width
pd.set_option("display.width", 1000)

In [2]:
!pip install sktime

  Using cached sktime-0.38.4-py3-none-any.whl.metadata (35 kB)
  Using cached scikit_base-0.12.4-py3-none-any.whl.metadata (8.8 kB)
Using cached sktime-0.38.4-py3-none-any.whl (35.5 MB)
Using cached scikit_base-0.12.4-py3-none-any.whl (145 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sktime]2m1/2 [sktime]


In [3]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in pd-multiindex mtype
X, _ = load_italy_power_demand(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["total_power_demand"]
X.index.names = ["day_ID", "hour_of_day"]

In [4]:
X

total_power_demand
day_ID hour_of_day                    
0      0                     -0.710518
       1                     -1.183320
       2                     -1.372442
       3                     -1.593083
       4                     -1.467002
...                                ...
1095   19                     0.180490
       20                    -0.094058
       21                     0.729587
       22                     0.210995
       23                    -0.002542

[26304 rows x 1 columns]

In [5]:
from sktime.datasets import load_basic_motions

# load an example time series panel in pd-multiindex mtype
X, _ = load_basic_motions(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["accel_1", "accel_2", "accel_3", "gyro_1", "gyro_2", "gyro_3"]
X.index.names = ["trial_no", "timepoint"]

In [6]:
X.head()

accel_1   accel_2   accel_3    gyro_1    gyro_2    gyro_3
trial_no timepoint                                                            
0        0          0.079106  0.394032  0.551444  0.351565  0.023970  0.633883
         1          0.079106  0.394032  0.551444  0.351565  0.023970  0.633883
         2         -0.903497 -3.666397 -0.282844 -0.095881 -0.319605  0.972131
         3          1.116125 -0.656101  0.333118  1.624657 -0.569962  1.209171
         4          1.638200  1.405135  0.393875  1.187864 -0.271664  1.739182

In [7]:
X.loc[0, "gyro_1"]


timepoint
0     0.351565
1     0.351565
2    -0.095881
3     1.624657
4     1.187864
        ...   
95    0.039951
96   -0.029297
97    0.000000
98    0.000000
99   -0.007990
Name: gyro_1, Length: 100, dtype: float64

In [8]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in numpy mtype
X, _ = load_italy_power_demand(return_type="numpy3D")

In [9]:
X.shape

(1096, 1, 24)

In [10]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_new, _ = load_italy_power_demand(split="test", return_type="numpy3D")

In [11]:
X_train.shape


(67, 1, 24)

In [12]:
y_train.shape


(67,)

In [13]:
# step 3 - specify the classifier
from sktime.transformations.panel.rocket import Rocket
# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = Rocket(num_kernels=256)

# example 2 - custom distance:
# 3-nearest neighbour classifier with Euclidean distance (on flattened time series)
# (requires scipy)

In [14]:
clf.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 256, 'random_state': None}

In [15]:
clf.fit(X_train, y_train)


Rocket(num_kernels=256)

In [19]:
clf.get_fitted_params()


{'classes': array(['1', '2'], dtype='<U1'),
 'fit_time': 117,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=3),
 'n_classes': 2,
 'knn_estimator__classes': array(['1', '2'], dtype='<U1'),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 67,
 'knn_estimator__n_samples_fit': 67,
 'knn_estimator__outputs_2d': False}

In [20]:
y_pred = clf.predict(X_new)


In [21]:
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

(array(['1', '2'], dtype='<U1'), array([510, 519]))